Как базовый взял ноутбук отсюда https://www.kaggle.com/sakshigoyal7/credit-card-customers. Понравились оформление и визуализация. Из-за этого часть текста на английском. Пришлось исправлять ошибки и почти полностью переработать код.

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
#from category_encoders import TargetEncoder
import catboost as catb
from sklearn.metrics import precision_score, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
import dill

In [2]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [3]:
def evaluate_preds(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

# 2.2 Data loading<a id = "2.2" ></a>

In [4]:
df = pd.read_csv('BankChurners.csv')
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 22 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   Attrition_Flag                                                                                                                      10127 non-null  int64  
 1   Customer_Age                                                                                                                        10127 non-null  int64  
 2   Gender                                                                                                                              10127 non-null  object 
 3   Dependent_count                                                                  

In [5]:
df.Total_Relationship_Count.value_counts()

3    2305
4    1912
5    1891
6    1866
2    1243
1     910
Name: Total_Relationship_Count, dtype: int64

# 5 Разделение на train и test<a id = "5"><a>

In [6]:
df=df.drop(['CLIENTNUM'], axis=1)

In [7]:
Attrition_Flag_to_numbers = {'Existing Customer': 0, 'Attrited Customer': 1}
df['Attrition_Flag'] = df['Attrition_Flag'].map(Attrition_Flag_to_numbers)

In [8]:
x_RAW = df[df.columns[1:]]
y_RAW = df['Attrition_Flag']


In [9]:
x_RAW.shape

(10127, 21)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x_RAW,y_RAW,test_size = 0.3,
                                                                 random_state =42,
                                                                 stratify=y_RAW)

In [11]:
X_train.shape, y_train.shape

((7088, 21), (7088,))

Взял признаки, feature_importances которых выше 4%

In [12]:
features = ['Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Trans_Ct_log',
 'Percent_Total_Amt_Chng', 'Max_Trans', 'Total_Relationship_Count',
 'Total_Amt_Chng_Q4_Q1', 'Months_Inactive_12_mon', 'Total_Revolving_Bal']
target = 'Attrition_Flag'

In [13]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [14]:
class DoubleLogTransformer(BaseEstimator, TransformerMixin):
    """Apply given transformation."""
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[f"{self.key}_log"] = np.log(np.log(X[self.key]))
        return X


In [15]:
class DivisionTransformer(BaseEstimator, TransformerMixin):
    """Apply given transformation."""
    def __init__(self, key1, key2):
        self.key1 = key1
        self.key2 = key2

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        x1 = self._get_selection1(X)
        x2 = self._get_selection2(X)
        X[f"{self.key1}_{self.key2}"] = x1 / x2
        return X
    
    def _get_selection1(self, X):
        assert isinstance(X, pd.DataFrame)
        return X[self.key1]
    
    def _get_selection2(self, X):
        assert isinstance(X, pd.DataFrame)
        return X[self.key2]

In [16]:
class ArrayViev(BaseEstimator, TransformerMixin):
    
     def fit(self, x, y=None):
        return self

     def transform(self, x):
        print(x[:5])


Собрал кусок, ответственный за feature engineering

In [17]:
Total_Trans_Ct_log = Pipeline([
                ('DoubleLogTransformer', DoubleLogTransformer(key = 'Total_Trans_Ct'))
            ])
Percent_Total_Amt_Chng = Pipeline([
                ('DivisionTransformer', DivisionTransformer(key1='Total_Ct_Chng_Q4_Q1', key2 = 'Total_Trans_Ct'))
            ])
Mean_Trans = Pipeline([
                ('DivisionTransformer', DivisionTransformer(key1='Total_Trans_Amt', key2='Total_Trans_Ct'))
            ])
Max_Trans = Pipeline([
                ('DivisionTransformer', DivisionTransformer(key1='Total_Revolving_Bal', key2='Total_Trans_Amt_Total_Trans_Ct'))
            ])
Selected_Columns = Pipeline([
                ('selector', ColumnSelector(['Total_Trans_Amt', 'Total_Trans_Ct',
                'Total_Relationship_Count', 'Total_Amt_Chng_Q4_Q1',
                'Months_Inactive_12_mon', 'Total_Revolving_Bal',
                'Total_Trans_Ct_log', 'Total_Ct_Chng_Q4_Q1_Total_Trans_Ct',
                'Total_Revolving_Bal_Total_Trans_Amt_Total_Trans_Ct']))
            ])

In [18]:
pipeline = Pipeline([
    ('Mean_Trans', Mean_Trans),
    ('Max_Trans', Max_Trans),
    ('Total_Trans_Ct_log', Total_Trans_Ct_log),
    ('Percent_Total_Amt_Chng', Percent_Total_Amt_Chng),
    ('Selected_Columns', Selected_Columns),
    ('classifier', catb.CatBoostClassifier(class_weights =[1, 5],
                            silent = True,
                            random_state = 21,
                            #cat_features = ['Total_Relationship_Count'
                             #              , 'Months_Inactive_12_mon'],
                            eval_metric = 'Precision',
                            early_stopping_rounds = 40))
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('Mean_Trans',
                 Pipeline(steps=[('DivisionTransformer',
                                  DivisionTransformer(key1='Total_Trans_Amt',
                                                      key2='Total_Trans_Ct'))])),
                ('Max_Trans',
                 Pipeline(steps=[('DivisionTransformer',
                                  DivisionTransformer(key1='Total_Revolving_Bal',
                                                      key2='Total_Trans_Amt_Total_Trans_Ct'))])),
                ('Total_Trans_Ct_log',
                 Pipeline(steps=[('DoubleLogTransformer',
                                  DoubleL...
                 Pipeline(steps=[('selector',
                                  ColumnSelector(key=['Total_Trans_Amt',
                                                      'Total_Trans_Ct',
                                                      'Total_Relationship_Count',
                                                      'Total_Amt_Chng_

In [19]:
predictions = pipeline.predict_proba(X_test)[:, 1]

In [20]:
predictions

array([2.12002875e-05, 2.83550152e-05, 5.26397502e-04, ...,
       2.79601916e-03, 9.57632596e-02, 8.35610746e-02])

In [21]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.792680, F-Score=0.892, Precision=0.929, Recall=0.859


In [22]:
with open("Churning_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [23]:
X_test.to_csv('X_test.csv', index=False)

In [24]:
y_test.to_csv('y_test.csv', index=False)

In [25]:
def predict_proba_to_int(y_predicted_probs, threshold):
         return (y_predicted_probs > threshold)*1

In [26]:
# Нашли метки предсказания для лучшего порога
best_predict = predict_proba_to_int(predictions, thresholds[ix])

In [27]:
roc_auc_score(best_predict, y_test)

0.9509257113428607